In [6]:
import glob
import os

data_dir = '../TML-Database/'  

csv_files = sorted(glob.glob(os.path.join(data_dir, '*.csv')))
print("CSV files found:", csv_files)

CSV files found: ['../TML-Database\\1968.csv', '../TML-Database\\1969.csv', '../TML-Database\\1970.csv', '../TML-Database\\1971.csv', '../TML-Database\\1972.csv', '../TML-Database\\1973.csv', '../TML-Database\\1974.csv', '../TML-Database\\1975.csv', '../TML-Database\\1976.csv', '../TML-Database\\1977.csv', '../TML-Database\\1978.csv', '../TML-Database\\1979.csv', '../TML-Database\\1980.csv', '../TML-Database\\1981.csv', '../TML-Database\\1982.csv', '../TML-Database\\1983.csv', '../TML-Database\\1984.csv', '../TML-Database\\1985.csv', '../TML-Database\\1986.csv', '../TML-Database\\1987.csv', '../TML-Database\\1988.csv', '../TML-Database\\1989.csv', '../TML-Database\\1990.csv', '../TML-Database\\1991.csv', '../TML-Database\\1992.csv', '../TML-Database\\1993.csv', '../TML-Database\\1994.csv', '../TML-Database\\1995.csv', '../TML-Database\\1996.csv', '../TML-Database\\1997.csv', '../TML-Database\\1998.csv', '../TML-Database\\1999.csv', '../TML-Database\\2000.csv', '../TML-Database\\2001.cs

In [7]:
import pandas as pd

df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)

print(f"Loaded {len(df)} matches from {len(csv_files)} files.")
df.head()

Loaded 193723 matches from 59 files.


C:\Users\naras\AppData\Local\Temp\ipykernel_20084\619315942.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_list, ignore_index=True)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,1968-9343,Bloemfontein,Hard,64,A,19680108.0,NaN,V125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-9343,Bloemfontein,Hard,64,A,19680108.0,NaN,U011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-9343,Bloemfontein,Hard,64,A,19680108.0,NaN,SS02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-9343,Bloemfontein,Hard,64,A,19680108.0,NaN,S250,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-9343,Bloemfontein,Hard,64,A,19680108.0,NaN,R085,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.isnull().sum())

tourney_id                 0
tourney_name               0
surface                 1269
draw_size                431
tourney_level              0
tourney_date              15
match_num              23394
winner_id                 46
winner_seed           121438
winner_entry          177352
winner_name                0
winner_hand             3763
winner_ht              19448
winner_ioc              1070
winner_age              2395
winner_rank            24994
winner_rank_points     35767
loser_id                  66
loser_seed            155883
loser_entry           166101
loser_name                 0
loser_hand              4683
loser_ht               32291
loser_ioc               1538
loser_age               6913
loser_rank             28022
loser_rank_points      38594
score                    105
best_of                    0
round                      7
minutes                89145
w_ace                  86590
w_df                   86590
w_svpt                 86592
w_1stIn       

In [9]:
key_columns = ['winner_rank', 'loser_rank', 'surface', 'round']
df = df.dropna(subset=key_columns)

In [10]:
features = [
    'winner_rank', 'loser_rank', 'surface', 'round',
    'winner_age', 'winner_ht', 'loser_age', 'loser_ht'
]

In [11]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

cat_features = ['surface', 'round']
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = enc.fit_transform(df[cat_features])

X_num = df[['winner_rank', 'loser_rank', 'winner_age', 'winner_ht', 'loser_age', 'loser_ht']].values
X = np.hstack([X_num, X_cat])

In [12]:
y = (df['winner_rank'] < df['loser_rank']).astype(int)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
print(df[['winner_rank', 'loser_rank', 'winner_age', 'winner_ht', 'loser_age', 'loser_ht']].isnull().sum())

winner_rank        0
loser_rank         0
winner_age       251
winner_ht       7240
loser_age       1064
loser_ht       14323
dtype: int64


In [ ]:

from sklearn.ensemble import RandomForestClassifier
df['winner_rank'] = pd.to_numeric(df['winner_rank'], errors='coerce')
df['loser_rank'] = pd.to_numeric(df['loser_rank'], errors='coerce')
df['winner_age'] = pd.to_numeric(df['winner_age'], errors='coerce')
df['winner_ht'] = pd.to_numeric(df['winner_ht'], errors='coerce')
df['loser_age'] = pd.to_numeric(df['loser_age'], errors='coerce')
df['loser_ht'] = pd.to_numeric(df['loser_ht'], errors='coerce')

df['winner_age'] = df['winner_age'].fillna(df['winner_age'].median())
df['winner_ht'] = df['winner_ht'].fillna(df['winner_ht'].median())
df['loser_age'] = df['loser_age'].fillna(df['loser_age'].median())
df['loser_ht'] = df['loser_ht'].fillna(df['loser_ht'].median())

X_num = df[['winner_rank', 'loser_rank', 'winner_age', 'winner_ht', 'loser_age', 'loser_ht']].values
X = np.hstack([X_num, X_cat])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9956922888916045
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     11081
           1       1.00      1.00      1.00     21651

    accuracy                           1.00     32732
   macro avg       1.00      0.99      1.00     32732
weighted avg       1.00      1.00      1.00     32732



In [22]:
importances = model.feature_importances_
feature_names = ['winner_rank', 'loser_rank', 'winner_age', 'winner_ht', 'loser_age', 'loser_ht'] + list(enc.get_feature_names_out(['surface', 'round']))
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
print(importance_df)

           feature  importance
0      winner_rank    0.459458
1       loser_rank    0.433230
4        loser_age    0.028701
2       winner_age    0.027276
5         loser_ht    0.014193
3        winner_ht    0.014045
18        round_RR    0.003762
16       round_R32    0.002538
17       round_R64    0.002226
7     surface_Clay    0.001872
9     surface_Hard    0.001838
19        round_SF    0.001719
13        round_QF    0.001661
14      round_R128    0.001643
15       round_R16    0.001546
6   surface_Carpet    0.001478
12         round_F    0.001476
8    surface_Grass    0.001298
11        round_BR    0.000024
10       round_3rd    0.000017


In [23]:
wimbledon_df = df[df['tourney_name'] == 'Wimbledon']
print(f"Found {len(wimbledon_df)} Wimbledon matches.")

Found 6268 Wimbledon matches.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_wimbledon = wimbledon_df[['winner_rank', 'loser_rank', 'winner_age', 'winner_ht', 'loser_age', 'loser_ht']].values
y_wimbledon = (wimbledon_df['winner_rank'] < wimbledon_df['loser_rank']).astype(int)

wimbledon_model = RandomForestClassifier(n_estimators=100, random_state=42)
wimbledon_model.fit(X_wimbledon, y_wimbledon)

RandomForestClassifier(random_state=42)

In [27]:
def create_advanced_features(df):
    df['winner_recent_form'] = df.groupby('winner_id')['winner_rank'].rolling(5).mean().reset_index(0, drop=True)
    df['loser_recent_form'] = df.groupby('loser_id')['loser_rank'].rolling(5).mean().reset_index(0, drop=True)
    
    df['winner_grass_win_rate'] = df.groupby(['winner_id', 'surface'])['winner_rank'].transform(
        lambda x: (x < x.shift(1)).mean()
    )
    
    df['h2h_matches'] = df.groupby(['winner_id', 'loser_id']).cumcount()
    
    df['winner_tourney_wins'] = df.groupby(['winner_id', 'tourney_name']).cumcount()
    
    df['days_since_last_match'] = df.groupby('winner_id')['tourney_date'].diff().dt.days
    
    return df

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

def train_ensemble_model(X_train, y_train):
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
    xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
    
    ensemble = VotingClassifier(
        estimators=[
            ('rf', rf),
            ('gb', gb),
            ('xgb', xgb_model)
        ],
        voting='soft'
    )
    
    ensemble.fit(X_train, y_train)
    return ensemble

def optimize_hyperparameters(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
    
    grid_search = GridSearchCV(
        RandomForestClassifier(),
        param_grid,
        cv=5,
        scoring='accuracy'
    )
    
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

In [30]:
import numpy as np
from typing import List, Dict

class BracketSimulator:
    def __init__(self, model, player_stats: Dict):
        self.model = model
        self.player_stats = player_stats
        
    def simulate_match(self, player1_id: str, player2_id: str) -> float:
        """Simulate a single match and return win probability for player1"""
        p1_stats = self.player_stats[player1_id]
        p2_stats = self.player_stats[player2_id]
        
        features = self._prepare_match_features(p1_stats, p2_stats)
        return self.model.predict_proba(features)[0][1]
    
    def simulate_round(self, players: List[str]) -> List[str]:
        """Simulate a round of matches and return winners"""
        winners = []
        for i in range(0, len(players), 2):
            if i + 1 < len(players):
                p1, p2 = players[i], players[i + 1]
                prob = self.simulate_match(p1, p2)
                winner = p1 if prob > 0.5 else p2
                winners.append(winner)
        return winners
    
    def simulate_tournament(self, seeds: List[str]) -> str:
        """Simulate entire tournament and return champion"""
        current_round = seeds
        while len(current_round) > 1:
            current_round = self.simulate_round(current_round)
        return current_round[0]
    
    def monte_carlo_simulation(self, seeds: List[str], n_simulations: int = 1000) -> Dict[str, float]:
        """Run multiple simulations to get win probabilities"""
        wins = {player: 0 for player in seeds}
        for _ in range(n_simulations):
            champion = self.simulate_tournament(seeds)
            wins[champion] += 1
        
        return {player: wins[player]/n_simulations for player in seeds}